In [53]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [54]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [55]:
df = pd.read_sql('SELECT * FROM marketsmojorecos', con=database_connection)

In [56]:
start_date = '2016-01-01'
end_date = '2016-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [57]:
mask = (df['predict_date'] > date_object1) & (df['predict_date'] <= date_object2)

# 2016 data

In [58]:
df1 = df.loc[mask]
df1 = df1.reset_index()
del df1['level_0']
del df1['srno']
del df1['index']

In [59]:
print("No of Brokers in 2016 :",df1['broker'].nunique()) 
s1=len(df1)

No of Brokers in 2016 : 61


In [60]:
df3=pd.DataFrame()
for i in range(0,5198): 
    sym=df1.at[i, 'company_name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    sqlrun = "SELECT SYMBOL FROM symbolsmapped WHERE recoName = '"+sym+"'"
    df3 = df3.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)

In [61]:
df4 =pd.concat([df1,df3],axis=1)
df4

broker          company_name  current_price  \
0             KRChoksey          Force Motors        1199.90   
1           AUM Capital   Narayana Hrudayalay         239.00   
2              Religare            ICICI Bank         390.15   
3     Enochventures.com      Parag Milk Foods         137.65   
4             Edelweiss  Kewal Kiran Clothing        1000.00   
...                 ...                   ...            ...   
5193        Axis Direct     Cox & Kings India           3.61   
5194      Dolat Capital               Mphasis         966.20   
5195      Angel Broking  Dewan Hsg Fin. Corp.          47.15   
5196      Angel Broking   MBL Infrastructures           5.57   
5197    ICICIdirect.com         Ultratech Cem        3883.75   

     recomended_buying predict_date target_price      SYMBOL  
0              3497.00   2016-10-03      4820.00    FORCEMOT  
1               325.55   2016-09-27       400.00          NH  
2               217.73   2016-08-04       181.82   ICICIBANK  
3                329.6   2016-07-12       425.00   PARAGMILK  
4              1796.00   2016-05-25      2039.00        KKCL  
...                ...          ...          ...         ...  
5193            179.00   2016-04-06       338.00       CKFSL  
5194            454.00   2016-04-05       620.00     MPHASIS  
5195            189.00   2016-04-05       270.00        DHFL  
5196            154.00   2016-04-04       285.00    MBLINFRA  
5197           3237.00   2016-04-01      3600.00  ULTRACEMCO  

[5198 rows x 7 columns]

In [62]:
#dftest = df4[0:100]
#dftest

In [63]:
uniqueBrokers = df4.broker.unique()
uniqueBrokers

array(['KRChoksey', 'AUM Capital', 'Religare', 'Enochventures.com',
       'Edelweiss', 'Motilal Oswal', 'Kotak Securities', 'Angel Broking',
       'Geojit Financial Services', 'Centrum', 'IIFL', 'ICICIdirect.com',
       'Axis Direct', 'Moneylicious', 'CD Equisearch', 'Nirmal Bang',
       'Arihant Capital Markets', 'Prabhudas Lilladher',
       'Wallfort Financial', 'HDFC Securities', 'BP Wealth',
       'Rudra Shares & Stocks Brokers', 'GEPL Capital', 'Dynamic Levels',
       'Choice Equity Broking', 'Right Horizons',
       'Systematix Shares & Stocks', 'East India Securities Ltd',
       'Sethi Finmart', 'Maximus Securities', 'Ajcon Global',
       'India Nivesh', 'Way2Wealth', 'YES Securities', 'AnandRathi',
       'SPA Securities', 'Khambatta Securities', 'Reliance Securities',
       'Dolat Capital', 'LKP Securities Limited',
       'Cholamandalam Securities', 'JM Financial', 'Ashika Stock Broking',
       'Antique Stock Broking', 'Karvy Stock Broking',
       'Blue Ocean Stra

In [64]:
#uniqueBrokers = dftest.broker.unique()
#uniqueBrokers

In [65]:
del brokerPerformace
brokerPerformace = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
#brokerPerformace = brokerPerformace.append({'Name':'a','Hit Ratio':'b','Avg. Growth':'c'}, ignore_index=True)
brokerPerformace

Empty DataFrame
Columns: [Name, Hit Ratio, Avg. Growth, Total Recos]
Index: []

In [68]:
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['broker'] == brokerName]
    #print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        #print(type(sym))
        if type(sym) == str :
            mydate1 = row['predict_date']#prediction date
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            dbyear1 = "nse"+str(mydate1.year)
            dbyear2 = "nse"+str(mydate2.year)
        
            sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
            del dfprices1['index']
        
            sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
            dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
            del dfprices2['index']
            dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)

            targetp = float(row['target_price'])
        
            #print("SYMBOL : ",sym)
            dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
            total = total + 1
            
            if not dfhits.empty :
                hits = hits + 1
            
            recbuying = float(row['recomended_buying'])
            maxval = dfprices['Closeval'].max()
            growthperc = (maxval - recbuying) *100 / recbuying
            #print('Growth : ',growthperc)
            if np.isnan(growthperc):
                total = total-1
            else:    
                growthSum = growthSum + growthperc 
    if total != 0:
        hitRatio = hits / total
        avgGrowth = growthSum / total
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformace = brokerPerformace.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

Broker :  KRChoksey HitRatio :  0.69377990430622 Avg. Growth :  83.56341526344873 Total Recos. :  209
Broker :  AUM Capital HitRatio :  0.7142857142857143 Avg. Growth :  69.17185149714541 Total Recos. :  21
Broker :  Religare HitRatio :  0.8038674033149171 Avg. Growth :  61.60740113010019 Total Recos. :  362
Broker :  Enochventures.com HitRatio :  0.43333333333333335 Avg. Growth :  42.22152062569151 Total Recos. :  30
Broker :  Edelweiss HitRatio :  0.7563025210084033 Avg. Growth :  147.17709302181296 Total Recos. :  476
Broker :  Motilal Oswal HitRatio :  0.701171875 Avg. Growth :  145.85051471518298 Total Recos. :  512
Broker :  Kotak Securities HitRatio :  0.8217054263565892 Avg. Growth :  672.5621003878041 Total Recos. :  129
Broker :  Angel Broking HitRatio :  0.5754716981132075 Avg. Growth :  77.82740811297978 Total Recos. :  106
Broker :  Geojit Financial Services HitRatio :  0.8317307692307693 Avg. Growth :  273.5192324545538 Total Recos. :  208
Broker :  Centrum HitRatio :  0.

In [69]:
brokerPerformace

Name  Hit Ratio  Avg. Growth  \
0                                 KRChoksey   0.693780    83.563415   
1                               AUM Capital   0.714286    69.171851   
2                                  Religare   0.803867    61.607401   
3                         Enochventures.com   0.433333    42.221521   
4                                 Edelweiss   0.756303   147.177093   
5                             Motilal Oswal   0.701172   145.850515   
6                          Kotak Securities   0.821705   672.562100   
7                             Angel Broking   0.575472    77.827408   
8                 Geojit Financial Services   0.831731   273.519232   
9                                   Centrum   0.733906    51.254127   
10                                     IIFL   0.777778   809.810034   
11                          ICICIdirect.com   0.788204   117.167532   
12                              Axis Direct   0.813808    68.267397   
13                             Moneylicious   0.800000    68.750591   
14                            CD Equisearch   0.822581    96.329249   
15                              Nirmal Bang   0.678571    77.877706   
16                  Arihant Capital Markets   0.902439    63.541592   
17                      Prabhudas Lilladher   0.733119   349.429833   
18                       Wallfort Financial   0.000000     6.595745   
19                          HDFC Securities   0.850000   696.216542   
20            Rudra Shares & Stocks Brokers   0.666667    30.105171   
21                             GEPL Capital   0.750000    33.289409   
22                           Dynamic Levels   0.873684    79.585822   
23                    Choice Equity Broking   1.000000    16.718555   
24                           Right Horizons   0.800000    47.203298   
25               Systematix Shares & Stocks   1.000000    46.248013   
26                East India Securities Ltd   0.750000   183.033332   
27                            Sethi Finmart   0.333333    55.622374   
28                       Maximus Securities   1.000000    36.608696   
29                             Ajcon Global   0.400000   193.743186   
30                             India Nivesh   0.785714   116.696415   
31                               Way2Wealth   0.666667    37.243390   
32                           YES Securities   0.833333    33.730124   
33                               AnandRathi   0.560976    91.983833   
34                           SPA Securities   0.641304    54.649063   
35                     Khambatta Securities   0.720000    32.130966   
36                      Reliance Securities   0.741497    47.158576   
37                            Dolat Capital   0.589286    48.099379   
38                   LKP Securities Limited   0.000000    14.024422   
39                 Cholamandalam Securities   0.000000     1.401596   
40                             JM Financial   0.711111    81.003349   
41                     Ashika Stock Broking   1.000000    18.785868   
42                    Antique Stock Broking   1.000000   335.861806   
43                      Karvy Stock Broking   0.869565   114.379921   
44            Blue Ocean Strategic Advisors   0.666667    55.173484   
45                                Sharekhan   0.476190    20.193067   
46                     Precision Technicals   0.666667    46.199319   
47                    Investment Imperative   1.000000    92.126522   
48      Avinash Gorakshakar - Market Expert   1.000000    11.930792   
49                           Hem Securities   1.000000    56.371448   
50                        Powermywealth.com   0.750000   101.610979   
51                          Shahina Mukadam   0.642857    14.969965   
52  Equinomics Research & Advisory Pvt. Ltd   1.000000    88.878049   
53                          Prakashgaba.com   0.600000    30.687261   
54                  Asian Market Securities   1.000000    71.020888   
55                            Elara Capital   1.000000    81.53

In [71]:
brokerPerformace.to_sql(con=database_connection, name='brokerperformance2016', if_exists='append')